## FBio EMG SENSOR 


In [1]:
import dash
from dash.dependencies import Output, Input
from dash import html, dcc
import plotly
import random
import pandas as pd
import plotly.graph_objs as go
from collections import deque


df = pd.read_csv("emg1_data.csv")
y_data = df["Raw Emg"].tolist()
# Initialize the deques with a maximum length
X = deque(maxlen=100)
X.append(1)
Y = deque(maxlen=100)
Y.append(1)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(
    [
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=100  # Update every 1 second
        ),
    ]
)

# Callback to update the graph
@app.callback(
    Output('live-graph', 'figure'),
    Input('graph-update', 'n_intervals')  # Use 'n_intervals' instead of events
)
def update_graph_scatter(n):
    # Update X and Y data
    X.append(X[-1] + 1)
    Y.append(y_data.pop())

    # Create the scatter plot data
    data = go.Scatter(
        x=list(X),
        y=list(Y),
        name='Scatter',
        mode='lines+markers'
    )

    # Define the layout with updated x and y axis ranges
    layout = go.Layout(
        xaxis=dict(range=[min(X), max(X)]),
        yaxis=dict(range=[0, 2])
    )

    # Return the updated figure
    return {'data': [data], 'layout': layout}

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import numpy as np
import biosppy.signals.emg as emg
import matplotlib.pyplot as plt

# Generate a random EMG-like signal array of size 1000
emg_array = np.random.normal(0, 0.05, 1000)  # mean 0, small std dev

# Process the signal using Biosppy
# Assuming the actual sampling rate is 1000 Hz
out = emg.emg(signal=emg_array, sampling_rate=1000, show=True)

# Extract relevant features
filtered_signal = out['filtered']  # The filtered EMG signal
onsets = out['onsets']  # Detected onsets of muscle activation

print("EMG processing complete.")

In [ ]:
import biosppy.signals.emg
import serial
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.signal import stft
from aux_functions import moving_average, apply_bandpass_filter, read_csv_as_emg_array, export_emg_features_to_csv, export_raw_emg_to_csv, compute_mdf, compute_mnf


# ---------- Data Acquisition ----------


# ---------- Signal Processing ----------
emg_array = read_csv_as_emg_array("emg1_data.csv")
#emg_array = np.array(emg_data)

emg_processed = biosppy.signals.emg.emg(signal=emg_array, sampling_rate=1000, show=False)
filtered_emg = apply_bandpass_filter(emg_array, lowcut=20, highcut=400, fs=1000, order=6)
rectified_emg = np.abs(filtered_emg)
smoothed_signal = moving_average(rectified_emg, window_size=3)



# ---------- STFT Calculation ----------
fs = 1000
nperseg = 1024

frequencies, t_stft, Zxx = stft(smoothed_signal, fs, nperseg=nperseg)


# Feature Extraction (Mean and Median Frequency)

# Mean frequency [MNF]

power_spectrum_stft = np.abs(Zxx) ** 2
mean_freq = compute_mnf(frequencies,power_spectrum_stft)
# Median frequency [MDF]

median_freq= compute_mdf(power_spectrum_stft)

# ---------- Plotting ----------
plt.figure(figsize=(14, 8))

plt.subplot(3,1,1)
plt.plot(emg_array, label="Raw Emg", color="blue", alpha=0.6)
plt.plot(filtered_emg, label="Filtered Emg", color="purple")
plt.title("Raw and Filtered Emg")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (mV)")
plt.grid()

plt.subplot(3,1,2)
plt.plot(rectified_emg, label="Rectified", color="red")
plt.title("Rectified Emg")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (mV)")
plt.grid()

plt.subplot(3,1,3)
plt.plot(smoothed_signal, label="Smoothed Signal", color="green")
plt.title("Smoothed Emg")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (mV)")
plt.grid()

plt.legend()
plt.tight_layout()
plt.show()

# Results Figure

plt.subplot(2,2,1)
plt.plot(mean_freq, label="Mean Frequency", color="red")
plt.title("Mean Frequency")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.grid()

plt.subplot(2,2,2)
plt.plot(median_freq, label="median frequency", color="green")
plt.title("Median Frequency")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.grid()

plt.subplot(2,1,2)
plt.plot(smoothed_signal, label="Filtered Emg", color="blue")
plt.title("Filtered Emg")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (mV)")
plt.grid()

plt.legend()
plt.tight_layout()
plt.show()

export_emg_features_to_csv(emg_array, mean_freq, median_freq, "emg_features.csv")
export_raw_emg_to_csv(emg_array,"raw_emg.csv")

